In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from utils.lof import LOF
from utils.metrics import euclidean, elastic_euclidean, _4dm_euclidean
from utils import get_table_players

In [3]:
from utils import Dataset

In [4]:
euclidean_regularized = elastic_euclidean(1, 0.3, 0.7)
fourdm_euclidean = _4dm_euclidean(1, 1)

In [5]:
_4dm4_impute = Dataset('4dm4_impute.db')

In [6]:
_4dm4_data = _4dm4_impute.select('scores', columns=['player_name', 'round', 'beatmap_type', 'beatmap_tag', 'score'], where={
    'beatmap_type': '\"LN\"'
})

In [7]:
_4dm4_data['score_logit'] = _4dm4_data['score'].apply(lambda x: np.log(x / (1e6 - x)))

In [8]:
_4dm4_data = _4dm4_data.drop('score', axis=1)

_4dm4_data.head()

,player_name,round,beatmap_type,beatmap_tag,score_logit
0,drag_,Q,LN,1,3.310021
1,JPBenedictGame,Q,LN,1,3.143908
2,iVALORUS,Q,LN,1,1.976354
3,BL5,Q,LN,1,3.635748
4,-Tebo-,Q,LN,1,3.915976


In [9]:
_4dm4_table = get_table_players(_4dm4_data)

In [10]:
standardized_4dm = pd.DataFrame(index=_4dm4_table.index)

In [11]:
for col in _4dm4_table.columns:
    mu = np.mean(_4dm4_table[col]); sig = np.std(_4dm4_table[col], ddof=1)
    standardized_4dm[col] = _4dm4_table[col].apply(lambda x: (x - mu) / sig)

In [12]:
standardized_4dm.head()

,Q_LN_1,RO32_LN_1,RO32_LN_2,RO32_LN_3,RO16_LN_1,RO16_LN_2,RO16_LN_3,QF_LN_1,QF_LN_2,QF_LN_3,SF_LN_1,SF_LN_2,SF_LN_3,F_LN_1,F_LN_2,F_LN_3,GF_LN_2,GF_LN_3,GF_LN_4
drag_,-0.382330,-1.300925,0.206909,-0.604581,-0.848575,-0.261085,-0.465997,-0.090186,-0.486272,-0.134528,-0.327734,-0.136475,-0.539713,-0.581798,-0.915829,-0.737081,-0.150882,-1.008905,-0.013753
JPBenedictGame,-0.603208,0.046728,-0.671629,-0.194652,-0.763800,-1.300649,-0.406536,-0.721721,0.389262,0.395112,-0.563624,-0.673075,-0.392264,-0.014715,-0.356454,-0.500414,-1.191343,-0.832129,-1.216027
iVALORUS,-2.155685,-1.622395,-1.877080,-1.268404,-2.217579,-2.391927,-1.424699,-1.462251,-1.364542,-2.086458,-1.871982,-2.071781,-2.582134,-1.925363,-2.117740,-2.115094,-1.632326,-1.403306,-1.762647
BL5,0.050783,-0.963618,0.688904,0.159091,0.246457,0.142597,0.142878,-0.158247,-0.389706,0.329919,-0.242365,-0.356498,0.158159,0.398619,-0.435489,-0.401048,-0.313025,-0.185097,-0.247861
-Tebo-,0.423398,0.112483,0.729382,0.311170,0.005301,0.368200,-0.471235,0.332999,0.820314,0.397010,0.137734,0.231068,0.672269,0.810095,0.770611,0.665358,-0.174240,0.280344,-0.223886


In [13]:
lof_euclid = LOF(2, euclidean_regularized)

In [14]:
lof_values = lof_euclid.fit_predict(standardized_4dm.values)

In [15]:
lof_4dm = pd.DataFrame(index=standardized_4dm.index)

lof_4dm['lof'] = lof_values

In [16]:
lof_4dm.loc['shokoha']

lof    0.927266
Name: shokoha, dtype: float64

In [17]:
lof_4dm.loc['Ricizus']

lof    0.993247
Name: Ricizus, dtype: float64

In [18]:
lof_4dm.sort_values(ascending=False, by="lof").head(16)

,lof
dsdsdd,1.874758
oliverq,1.725567
Maykee kee,1.628798
diamondBIaze,1.619234
Doryoku,1.545020
SimoHero,1.513760
StayHighKids,1.503549
Zergh,1.472578
wolfpup08,1.430626
[LS]Byte,1.407652
